In [1]:
"""prep_eu_data.py
======================================================
Processes raw economic data for Dynare DSGE model estimation.

Key transformations based on supervisor feedback:
- Logs for GDP, deflator, nominal wages.
- HP filter for output gap (cyclical component directly used, approx. mean zero).
- Calculates annualized nominal price inflation and nominal wage inflation.
- Scales interest rates.
- Demeans inflation rates and interest rate to ensure they are deviations around zero.
- Saves to eu_data.mat.

CRITICAL CHANGE (original): Wage inflation is now calculated as NOMINAL wage inflation.
SUPERVISOR FEEDBACK INCORPORATED:
  - y_obs (output gap): Use HP filter cyclical component directly (no further demeaning).
  - Other observables (inflations, interest rate): Continue to demean after scaling.
======================================================
"""

import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.io import savemat
from pathlib import Path

# ------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------
FILE = Path("Data_EU_Base.xlsx")  # Input Excel file name
SHEET = 0                         # Sheet index or name in the Excel file
LAMBDA = 1600                     # HP-filter smoothing parameter (for quarterly data)

# Column names in your Excel file corresponding to the economic series
CODES = {
    "gdp": "YER",      # Real GDP
    "defl": "YED",     # GDP Deflator
    "wage": "WIN",     # Nominal Wages
    "rate": "STN"      # Short-term nominal interest rate
}
# ------------------------------------------------------------------

def hp_filter(x, lamb):
    T = len(x)
    I_mat = sparse.eye(T)
    D_mat = sparse.diags([np.ones(T), -2 * np.ones(T), np.ones(T)], [0, 1, 2], shape=(T - 2, T))
    trend_component = spsolve(I_mat + lamb * (D_mat.T @ D_mat), x)
    cycle_component = x - trend_component
    return cycle_component, trend_component

# 1) Read data and convert relevant columns to numeric ----------------
print(f"[prep] Reading data from: {FILE.name}, sheet: {SHEET}")
try:
    df = pd.read_excel(FILE, sheet_name=SHEET, dtype=object) # Read as object initially
except FileNotFoundError:
    print(f"[prep] ERROR: File not found: {FILE.name}")
    raise
except Exception as e:
    print(f"[prep] ERROR reading Excel file: {e}")
    raise

missing_cols = [col_code for col_code in CODES.values() if col_code not in df.columns]
if missing_cols:
    print(f"[prep] ERROR: Missing required columns in {FILE.name}: {missing_cols}")
    raise KeyError(f"Missing columns: {missing_cols}")

for series_key, col_name in CODES.items():
    # Ensure the column is treated as string before attempting string operations
    # This handles cases where pandas might have inferred a numeric type for some cells
    # or if the column is mixed.
    if df[col_name].dtype == 'object': # It's already object/string, or mixed
        # Further check if any string replacement is actually needed
        # This avoids unnecessary conversion if it's already clean numeric-like strings or actual numbers
        # We can try to convert to numeric first. If it fails widely, then it likely has commas.
        # A more direct way is to convert to string, then replace.
        try:
            # Convert the entire column to string type first to ensure .str accessor works
            df[col_name] = df[col_name].astype(str)
            # Now apply string replacement
            df[col_name] = df[col_name].str.replace(',', '.', regex=False)
        except AttributeError:
            # This might happen if the column was already fully numeric and astype(str) wasn't needed
            # or if there are other non-string types that .str can't handle after astype(str)
            # (e.g. if it contained datetime objects that became strings like '2023-01-01 00:00:00')
            # For our specific case (expecting numbers or strings representing numbers),
            # this path is less likely if dtype=object was effective.
            print(f"[prep] INFO: Column '{col_name}' might not be purely string-like or already processed. Skipping replace for safety.")
            pass # If it's already fine or not string, to_numeric will handle it or coerce errors

    # Convert to numeric, coercing errors for anything that couldn't be made numeric
    df[col_name] = pd.to_numeric(df[col_name], errors="coerce")

    if df[col_name].isnull().any():
        # Check if NaNs were present BEFORE this conversion attempt to pinpoint the source
        # For now, this warning is after to_numeric
        print(f"[prep] WARNING: NaNs found or introduced in column '{col_name}' after numeric conversion. Check raw data and formatting.")

gdp_raw = df[CODES["gdp"]].to_numpy()
defl_raw = df[CODES["defl"]].to_numpy()
wage_raw = df[CODES["wage"]].to_numpy()
rate_raw = df[CODES["rate"]].to_numpy()

# 2) Data cleaning and initial transformations -------------------------
mask = (gdp_raw > 0) & (defl_raw > 0) & (wage_raw > 0) & \
       ~np.isnan(gdp_raw) & ~np.isnan(defl_raw) & ~np.isnan(wage_raw) & ~np.isnan(rate_raw)

if not np.any(mask):
    print("[prep] ERROR: No valid data points after initial filtering.")
    raise ValueError("No valid data after initial mask.")

print(f"[prep] Initial observations: {len(gdp_raw)}")
print(f"[prep] Observations after filtering for positive loggables & non-NaNs: {np.sum(mask)}")

log_gdp = np.log(gdp_raw[mask])
log_defl = np.log(defl_raw[mask])
log_wage_nominal = np.log(wage_raw[mask])
rate_annual_pct = rate_raw[mask]

# 3) Calculate output gap (HP-filtered log real GDP) -----------------
y_gap_unaligned, _ = hp_filter(log_gdp, LAMBDA) # y_gap_unaligned is the cyclical component

# 4) Calculate inflation rates (annualized percentage points) --------
pinf_annual_pct = 400 * np.diff(log_defl)
winf_nominal_annual_pct = 400 * np.diff(log_wage_nominal)

# 5) Align series due to differencing --------------------------------
y_gap_aligned = y_gap_unaligned[1:] # This is the cyclical component from HP filter
rate_annual_pct_aligned = rate_annual_pct[1:]

T_obs = len(pinf_annual_pct)
assert len(winf_nominal_annual_pct) == T_obs, "Wage inflation length mismatch"
assert len(y_gap_aligned) == T_obs, "Output gap length mismatch"
assert len(rate_annual_pct_aligned) == T_obs, "Interest rate length mismatch"

print(f"[prep] Final number of observations for estimation: {T_obs}")

# 6) Scale and prepare for Dynare (incorporating supervisor feedback) -----
# Output Gap: Use HP-filtered cyclical component directly.
# The HP filter's cyclical component is already (approximately) mean zero.
y_obs = y_gap_aligned
print(f"[prep] Output gap (y_obs) using HP-filter cyclical component directly. Mean before saving: {y_obs.mean():.4e}")

# Price Inflation (convert from annualized % to quarterly decimal, then demean)
pinf_quarterly_decimal = pinf_annual_pct / 400
pinf_obs = pinf_quarterly_decimal - pinf_quarterly_decimal.mean()

# Nominal Wage Inflation (convert from annualized % to quarterly decimal, then demean)
winf_quarterly_decimal = winf_nominal_annual_pct / 400
w_obs = winf_quarterly_decimal - winf_quarterly_decimal.mean()

# Interest Rate (convert from % p.a. to quarterly decimal, then demean)
rate_quarterly_decimal = rate_annual_pct_aligned / 400
r_obs = rate_quarterly_decimal - rate_quarterly_decimal.mean()

# 7) Save to .mat file for Dynare ------------------------------------
mat_file_name = "eu_data.mat" # Ensure this is the name your .mod file expects
savemat(mat_file_name, {
    "y_obs": y_obs.reshape(-1, 1),
    "w_obs": w_obs.reshape(-1, 1),
    "r_obs": r_obs.reshape(-1, 1),
    "pinf_obs": pinf_obs.reshape(-1, 1)
})
print(f"\n[prep] Data saved to: {mat_file_name}")

# 8) Preview processed data ------------------------------------------
print("\n[prep] First 5 observations of processed data (for Dynare):")
preview_df = pd.DataFrame({
    "y_obs": y_obs[:5],
    "w_obs": w_obs[:5],
    "r_obs": r_obs[:5],
    "pinf_obs": pinf_obs[:5]
})
print(preview_df)

# ------------------------------------------------------------------
# 9) Save long-run means for future back-transformation
# ------------------------------------------------------------------
mean_file = "eu_data_means.mat"
savemat(mean_file, {
    # Quarterly decimal (not annualised) means
    "pif_mean": pinf_quarterly_decimal.mean(),   # price inflation mean (q)
    "piw_mean": winf_quarterly_decimal.mean(),   # wage inflation mean (q)
    "r_mean"  : rate_quarterly_decimal.mean()    # interest-rate mean  (q)
})
print(f"[prep] Long-run means saved to: {mean_file}")


print("\n[prep] Means of processed data (should be close to zero for demeaned series):")
print(f"  Mean y_obs:    {y_obs.mean():.4e} (HP-filtered gap, not explicitly demeaned here)")
print(f"  Mean w_obs:    {w_obs.mean():.4e} (explicitly demeaned)")
print(f"  Mean r_obs:    {r_obs.mean():.4e} (explicitly demeaned)")
print(f"  Mean pinf_obs: {pinf_obs.mean():.4e} (explicitly demeaned)")

print("\n[prep] Standard deviations of processed data (for prior consideration):")
print(f"  StdDev y_obs:    {y_obs.std():.4f}")
print(f"  StdDev w_obs:    {w_obs.std():.4f}")
print(f"  StdDev r_obs:    {r_obs.std():.4f}")
print(f"  StdDev pinf_obs: {pinf_obs.std():.4f}")

print("\n[prep] Data preparation complete.")

[prep] Reading data from: Data_EU_Base.xlsx, sheet: 0
[prep] Initial observations: 214
[prep] Observations after filtering for positive loggables & non-NaNs: 214
[prep] Final number of observations for estimation: 213
[prep] Output gap (y_obs) using HP-filter cyclical component directly. Mean before saving: 5.3850e-05

[prep] Data saved to: eu_data.mat

[prep] First 5 observations of processed data (for Dynare):
      y_obs     w_obs     r_obs  pinf_obs
0 -0.002816  0.029481  0.006018  0.009660
1 -0.001218  0.023749  0.005132  0.003964
2  0.000461  0.020372  0.004233  0.000339
3 -0.011625  0.018756  0.002419  0.015653
4 -0.009197  0.006117  0.000973  0.000414
[prep] Long-run means saved to: eu_data_means.mat

[prep] Means of processed data (should be close to zero for demeaned series):
  Mean y_obs:    5.3850e-05 (HP-filtered gap, not explicitly demeaned here)
  Mean w_obs:    2.6062e-18 (explicitly demeaned)
  Mean r_obs:    0.0000e+00 (explicitly demeaned)
  Mean pinf_obs: 1.0425e-18